In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=d607b11f53f962696d33f79e8091cee2954b6f53924b6830191287b4b7f24109
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F3 '''
def F3(X):
    f = bn.F3()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.199 ,random_state=0 , l1_ratio=0.95, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 69.69 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_3_2000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_3_400Samples.csv').iloc[:,-1]
path = "train_latent_100D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_100D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=916871, Mon Apr 13 13:55:25 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 1.301153246997964e+04 1.0e+00 2.55e-01  3e-01  3e-01 0:00.0
    2     30 1.301158009527335e+04 1.0e+00 2.47e-01  2e-01  2e-01 0:00.0
    3     45 1.301086810340010e+04 1.1e+00 2.38e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [13012.25281619] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [13011.58009527] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -9.386210611235496e+03 1.5e+00 3.76e-02  4e-02  4e-02 0:02.0
  200   3000 -1.080121268067034e+04 1.5e+00 2.50e-03  2e-03  2e-03 0:04.0
  300   4500 -1.080642176121269e+04 1.5e+00 1.63e-04  1e-04  2e-04 0:06.0
  400   6000 -1.080644489521813e+04 1.5e+00 9.11e-06  8e-06  9e-06 0:08.0
  500   7500 -1.080644494551885e+04 1.6e+00 5.71e-07  4e-07  5e-07 0:09.9
  600   9000 -1.080644494565319e+04 1.6e+00 3.97e-08  3e-08  4e-08 0:11.9
  700  10500 -1.080644494565418e+04 1.7e+00 2.92e-09  2e-09  2e-09 0:13.9
  800  12000 -1.080644494565418e+04 1.9e+00 1.43e-09  1e-09  1e-09 0:15.9
  900  13500 -1.080644494565418e+04 2.1e+00 9.30e-10  7e-10  8e-10 0:17.9
 1000  15000 -1.080644494565418e+04 2.3e+00 6.21e-10  4e-10  5e-10 0:20.0
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.080644494565418e+04 2.4e+00 6.23e-10  4e-10  5e-10 0:21.9
 1200  18000 -1.080644494565418e+04 2.6e+00 4.75e-10  3e-10  4e-10 0:23.9
 1300  19500 -1.080644494565418e+04 2.8e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [12551.07538939] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=12551.075389, sigma=2.53e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=12551.075389, sigma=2.53e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

  100   1500 -1.645305739856714e+09 1.8e+00 5.94e+01  6e+01  6e+01 0:01.8
  121   1815 -4.585173602155110e+10 1.9e+00 2.72e+02  3e+02  3e+02 0:02.2
termination on tolfacupx=1000.0 (Mon Apr 13 13:56:28 2020)
final/bestever f-value = -4.391785e+10 -4.585174e+10
incumbent solution: [  548.81972416   498.60289987   276.69369157   899.23647636
  1593.6540423    432.60057762 -1287.48267414  -606.83841798 ...]
std deviations: [267.05198552 261.42964368 259.2500951  272.12095931 267.56063034
 259.32066313 273.48635407 267.37513006 ...]
Run : 1
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=848355, Mon Apr 13 13:56:28 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 1.300096277113439e+04 1.0e+00 2.54e-01  3e-01  3e-01 0:00.0
    2     30 1.296691687271940e+04 1.0e+00 2.44e-01  2e-01  2e-01 0:00.0
    3     45 1.292137234412502e+04 1.0e+00 2.39e-01  2e-01  2e-01 0:00.1


...

  100   1500 -1.193629311969568e+10 1.4e+00 1.83e+02  2e+02  2e+02 0:01.9
  106   1590 -2.595337241570750e+10 1.4e+00 2.88e+02  3e+02  3e+02 0:02.0
termination on tolfacupx=1000.0 (Mon Apr 13 14:25:38 2020)
final/bestever f-value = -2.665760e+10 -2.665760e+10
incumbent solution: [ -260.58619267   646.38825723   593.795688     959.10777663
   742.50462516  1073.02376523 -1460.70317134   -61.74345099 ...]
std deviations: [278.08528232 280.15283919 277.02102466 276.61806534 274.41249308
 273.04731344 286.59985983 276.66537167 ...]
Run : 29
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=939185, Mon Apr 13 14:25:38 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 1.299734831971565e+04 1.0e+00 2.53e-01  3e-01  3e-01 0:00.0
    2     30 1.299802793145686e+04 1.0e+00 2.43e-01  2e-01  2e-01 0:00.0
    3     45 1.297998881751063e+04 1.0e+00 2.39e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [13003.73384264] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [13007.08573422] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -9.471126550185494e+03 1.4e+00 3.94e-02  4e-02  4e-02 0:02.0
  200   3000 -1.080209581290412e+04 1.4e+00 2.09e-03  2e-03  2e-03 0:04.1
  300   4500 -1.080643044261738e+04 1.5e+00 1.45e-04  1e-04  1e-04 0:06.1
  400   6000 -1.080644489049202e+04 1.5e+00 9.03e-06  7e-06  8e-06 0:08.1
  500   7500 -1.080644494545115e+04 1.6e+00 6.90e-07  5e-07  6e-07 0:10.2
  600   9000 -1.080644494565315e+04 1.6e+00 4.12e-08  3e-08  4e-08 0:12.2
  700  10500 -1.080644494565417e+04 1.7e+00 3.47e-09  2e-09  3e-09 0:14.2
  800  12000 -1.080644494565418e+04 1.9e+00 1.63e-09  1e-09  1e-09 0:16.3
  900  13500 -1.080644494565418e+04 2.2e+00 1.46e-09  1e-09  1e-09 0:18.3
 1000  15000 -1.080644494565418e+04 2.4e+00 1.06e-09  7e-10  9e-10 0:20.3
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.080644494565418e+04 2.6e+00 9.34e-10  6e-10  8e-10 0:22.4
 1200  18000 -1.080644494565418e+04 2.8e+00 5.54e-10  4e-10  5e-10 0:24.4
 1300  19500 -1.080644494565419e+04 3.0e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [12551.07538939] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=12551.075389, sigma=2.54e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=12551.075389, sigma=2.54e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

  100   1500 -5.371079712995313e+09 1.5e+00 1.15e+02  1e+02  1e+02 0:01.8
  112   1680 -2.730015404556274e+10 1.5e+00 2.79e+02  3e+02  3e+02 0:02.1
termination on tolfacupx=1000.0 (Mon Apr 13 14:27:00 2020)
final/bestever f-value = -2.658185e+10 -2.730015e+10
incumbent solution: [-783.75804543  497.55599143 -422.11576663  328.32791625  671.4177232
  273.00914474   37.38396749 -836.65960194 ...]
std deviations: [269.97868473 268.37319211 272.78921402 277.23167401 268.53426538
 273.52520432 277.65878719 267.45466745 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F3_100_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F3(X_Values.iloc[i,:50])
    SVM_Fun [i] = F3(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F3(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([1949.35840572]), count=array([1])) 3.5476500677512417e-06
SVM
ModeResult(mode=array([2093.90594489]), count=array([1])) 207.7446470534542
ELN
ModeResult(mode=array([4.16920213e+41]), count=array([1])) 9.30395657076365e+59
